In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('../../../module/')

from keras2.models import Model
from keras2.layers import concatenate, Dense, Input, Flatten
from keras2.optimizers import Adam, Optimizer
import keras2.backend as K
import tensorflow as tf
import csv
import itertools
from util import *
import gym2
from rl2.agents import selfDDPGAgent, selfDDPGAgent2, selfDDPGAgent3
from rl2.memory import SequentialMemory

Using TensorFlow backend.
Using TensorFlow backend.


In [2]:
env = gym2.make('Linear-v1')
Q = .01 * np.eye(2)
R = .01

In [3]:
def critic_net(a_shape , s_shape):
    action_input = Input(a_shape)
    observation_input = Input(shape=(1,)+s_shape)
    flattened_observation = Flatten()(observation_input)
    x = concatenate([action_input, flattened_observation])
    x = Dense(16, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(1, activation="linear")(x)
    critic = Model(inputs=[action_input, observation_input], outputs=x)
    return (critic, action_input)

def value_net(s_shape):
    state_input = Input((1,)+s_shape)
    x = Flatten()(state_input)
    
    x = Dense(16, activation='relu')(x)
    x = Dense(16, activation='relu')(x)
    x = Dense(1, activation="linear")(x)
    value = Model(inputs=state_input, output=x)
    return value

def branch_actor(a_shape, s_shape):
    action_input = Input(shape=(1,)+s_shape)
    x = Flatten()(action_input) # 実質的なinput layer
    
    x1 = Dense(16, activation="relu")(x)
    x1 = Dense(16, activation="relu")(x1)
    x1 = Dense(1, activation="multiple_tanh")(x1) # action signal
    
    x2 = Dense(16, activation="relu")(x)
    x2 = Dense(16, activation="relu")(x2)
    x2 = Dense(1, activation="tau_output_large")(x2) # tau
    
    output = concatenate([x1, x2])
    actor = Model(inputs=action_input, outputs=output)
    return actor

In [4]:
actor = branch_actor((2,),(2,))
critic = critic_net((2,),(2,))[0]
critic.compile(Adam(lr=0.001, clipnorm=1.), loss='mean_squared_error')

In [5]:
alpha = 0.4
beta = 1.
dummy_optimizer = Optimizer()
def gradient_for_one_data(state, next_state, actor, critic):
    params = actor.trainable_weights  
    # q_func gradient tensor
    state_input = tf.constant([[state.tolist()]], tf.float32)
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    loss = -K.mean(critic_output_tensor)
    gradient_for_q = dummy_optimizer.get_gradients(loss, params)
    
    # discount factor gradient tensor
    discount = - tf.exp(- alpha * actor_output[0])
    gradient_for_d = dummy_optimizer.get_gradients(discount, params)
    next_state_input = tf.constant([[next_state.tolist()]], tf.float32)
    next_action = actor(next_state_input)
    next_value = critic([next_action, next_state_input])[0]
    gradient_for_d = [next_value * g for g in gradient_for_d]
    
    gradient_for_one_state = [K.add(gq, gd) for gq, gd in zip(gradient_for_q, gradient_for_d)]
    
    return gradient_for_one_state

def gradient(states, ts, next_states, actor, critic):
    batch_size = len(states)
    for i in range(batch_size):
        state, t, next_state = states[i], ts[i], next_states[i]
        discount = np.exp(- alpha * np.array(t))
        g_tensor = gradient_for_one_data(state, next_state, actor, critic)
        g_tensor = [discount * g for g in g_tensor]
        if i == 0:
            out = [tf.constant(0.)] * len(g_tensor)
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
        else:
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        g = sess.run(out) 
    return g

In [14]:
states = [np.array([1,2])]
next_states = [np.array([1,2])]
ts = np.random.uniform(0,10,2)
discounts = np.exp(- alpha * np.array(ts))
out = gradient(states,ts,states,actor,critic)

In [97]:
def gradient_for_one_data(state, next_state, actor, critic):
    params = actor.trainable_weights  
    # q_func gradient tensor
    state_input = tf.constant([[state.tolist()]], tf.float32)
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    loss = -K.mean(critic_output_tensor)
    gradient_for_q = dummy_optimizer.get_gradients(loss, params)
    
    # discount factor gradient tensor
    discount = - tf.exp(- alpha * actor_output[0][1])
    gradient_for_d = dummy_optimizer.get_gradients(discount, params)
    next_state_input = tf.constant([[next_state.tolist()]], tf.float32)
    next_action = actor(next_state_input)
    next_value = critic([next_action, next_state_input])[0][0]
    gradient_for_d = [next_value * g for g in gradient_for_d]
    
    gradient_for_one_state = [K.add(gq, gd) for gq, gd in zip(gradient_for_q, gradient_for_d)]
    
    return gradient_for_one_state

def gradient(states, ts, next_states, actor, critic):
    batch_size = len(states)
    for i in range(batch_size):
        state, t, next_state = states[i], ts[i], next_states[i]
        discount = np.exp(- alpha * t)
        g_tensor = gradient_for_one_data(state, next_state, actor, critic)
        g_tensor = [discount * g for g in g_tensor]
        if i == 0:
            out = [tf.constant(0.)] * len(g_tensor)
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
        else:
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
    func = K.function([], out)
    gg = func([[]])
    return gg


In [127]:
def gradient(states, ts, next_states, actor, critic):
    batch_size = len(states)
    for i in range(batch_size):
        state, t, next_state = states[i], ts[i], next_states[i]
        discount = np.exp(- alpha * t)
        g_tensor = gradient_for_one_data(state, next_state, actor, critic)
        g_tensor = [discount * g for g in g_tensor]
        if i == 0:
            out = [tf.constant(0.)] * len(g_tensor)
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
        else:
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
    func = K.function([], out)
    gg = func([[]])
    return gg

def gradient_function(actor, critic):
    params = actor.trainable_weights
    # placeholder
    state_input = tf.placeholder(tf.float32, shape=(None, 1, 2))
    ts_input = tf.placeholder(tf.float32, shape=(None, 1))
    values_input = tf.placeholder(tf.float32, shape=(None, 1))
    
    # q_func gradient tensor
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    obj1 = tf.multiply(K.exp(- alpha * ts_input), critic_output_tensor)
    loss1 = -K.sum(obj1)
    gradient_for_q = dummy_optimizer.get_gradients(loss1, params)
    
    # discount factor gradient tensor
    discount = tf.expand_dims(- tf.exp(- alpha * actor_output[:,1]), 1)
    weights = tf.multiply(K.exp(- alpha * ts_input), values_input) # param勾配ないならok
    obj2 = tf.multiply(weights, discount)
    loss2 = -K.sum(obj2)
    gradient_for_d = dummy_optimizer.get_gradients(loss2, params)
    
    gradient_tensor = [K.add(gq, gd) for gq, gd in zip(gradient_for_q, gradient_for_d)]
    func = K.function([state_input, ts_input, values_input], gradient_tensor)
    
    return func

def gradient2(states, ts, values, gradient_func):
    gradient = gradient_func([states, ts, values])
    return gradient


In [114]:
gradient2(states, ts, values, next_states, func)

[array([[ 0.        ,  1.0928483 ,  1.684546  ,  0.        ,  0.        ,
          2.0761619 ,  0.76795936,  0.        ,  0.        , -1.1276249 ,
         -0.24912782,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  1.8865546 ,  2.9079862 ,  0.        ,  0.        ,
          3.5840223 ,  1.3257074 ,  0.        ,  0.        , -1.9465883 ,
         -0.4300626 ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]], dtype=float32),
 array([ 0.        ,  0.7937062 ,  1.2234402 ,  0.        ,  0.        ,
         1.5078602 ,  0.5577481 ,  0.        ,  0.        , -0.8189634 ,
        -0.18093479,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ], dtype=float32),
 array([[ 0.        ,  0.        ,  0.        , -0.67222697,  0.        ,
         -0.6477427 ,  0.        ,  1.2680855 ,  0.        ,  0.        ,
          0.        ,  1.4286915 ,  0.94173014,  0.        ,  0.78489923,
          0.93

In [107]:
gradient(states, ts, next_states, actor, critic)

[array([[ 0.        ,  0.21638691,  0.3335446 ,  0.        ,  0.        ,
          0.4110856 ,  0.15205799,  0.        ,  0.        , -0.22327277,
         -0.04932799,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ],
        [ 0.        ,  0.40352675,  0.622007  ,  0.        ,  0.        ,
          0.7666084 ,  0.28356367,  0.        ,  0.        , -0.41636777,
         -0.09198875,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ]], dtype=float32),
 array([ 0.        ,  0.18713982,  0.2884624 ,  0.        , -0.        ,
         0.35552284,  0.13150568,  0.        ,  0.        , -0.19309498,
        -0.04266077, -0.        ,  0.        ,  0.        , -0.        ,
        -0.        ], dtype=float32),
 array([[ 0.        ,  0.        ,  0.        , -0.07362843,  0.        ,
         -0.0709467 ,  0.        ,  0.13889231,  0.        ,  0.        ,
          0.        ,  0.15648335,  0.10314688,  0.        ,  0.08596932,
          0.10

In [160]:
params = actor.trainable_weights
# placeholder
state_input = tf.placeholder(tf.float32, shape=(None, 1, 2))
ts_input = tf.placeholder(tf.float32, shape=(None, 1))
values_input = tf.placeholder(tf.float32, shape=(None, 1))


# discount factor gradient tensor
actor_output = actor(state_input)
discount = tf.expand_dims(- tf.exp(- alpha * actor_output[:,1]), 1)
weights = tf.multiply(K.exp(- alpha * ts_input), values_input) # param勾配ないならok
obj2 = tf.multiply(weights, discount)
loss2 = -K.sum(obj2)
gradient_for_d = dummy_optimizer.get_gradients(loss2, params)

loss3 = - tf.matmul(tf.transpose(weights), discount)[0][0]

f = K.function([state_input, ts_input, values_input], [loss2])
print(f([states, ts, values]))
f = K.function([state_input, ts_input, values_input], [loss3])
print(f([states, ts, values]))

[0.025434438]
[0.025434438]


In [207]:
states = np.random.randn(4, 1, 2)
next_states = np.random.randn(4, 1, 2)
ts = [[0], [2], [4], [7]]
actions = actor.predict_on_batch(next_states)
values = critic.predict_on_batch([actions, next_states])

In [193]:
# Qの方の微分は？ -同じ
def q1(actor, critic, states, ts, values):
    params = actor.trainable_weights  
    state_input = tf.placeholder(tf.float32, shape=(None, 1, 2))
    ts_input = tf.placeholder(tf.float32, shape=(None, 1))
    values_input = tf.placeholder(tf.float32, shape=(None, 1))
    
    # q_func gradient tensor
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    obj1 = tf.multiply(K.exp(- alpha * ts_input), critic_output_tensor)
    loss1 = -K.sum(obj1)
    gradient_for_q = dummy_optimizer.get_gradients(loss1, params)
    f = K.function([state_input, ts_input, values_input], gradient_for_q)
    return f([states, ts, values])


print(q1(actor, critic, states, ts, values)[0])

def g4one(actor, critic, state):
    params = actor.trainable_weights  
    # q_func gradient tensor
    state_input = tf.constant([[state.tolist()]], tf.float32)
    actor_output = actor(state_input)
    combined_input_tensor = [actor_output, state_input]
    critic_output_tensor = critic(combined_input_tensor)
    loss = -K.mean(critic_output_tensor)
    gradient_for_q = dummy_optimizer.get_gradients(loss, params)
    
    return gradient_for_q

def q2(actor, critic, states, ts, next_states):
    batch_size = len(states)
    for i in range(batch_size):
        state, t = states[i], ts[i][0]
        discount = np.exp(- alpha * t)
        g_tensor = g4one(actor, critic, state)
        g_tensor = [discount * g for g in g_tensor]
        if i == 0:
            out = [tf.constant(0.)] * len(g_tensor)
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
        else:
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
    func = K.function([], out)
    gg = func([[]])
    return gg

print(q2(actor, critic, states, ts, values)[0])


[[ 0.15820403 -0.42731363  0.15713947  0.29690138 -0.23312613  0.09578644
   0.01795874 -0.0581277  -0.25341547  0.27255696 -0.2521754   0.183599
   0.3632166  -0.6738815   0.08403675  0.12388417]
 [-0.08670373  0.10614868  0.00561079 -0.0467115   0.12622696  0.06298667
   0.00522941 -0.09002     0.03938799 -0.07473399  0.075108   -0.0292979
  -0.14273576  0.10621426 -0.01355848 -0.01956656]]
[[ 0.15820408 -0.42731363  0.15713947  0.29690138 -0.23312618  0.09578642
   0.01795873 -0.05812765 -0.25341544  0.2725569  -0.2521754   0.18359898
   0.3632166  -0.6738814   0.08403675  0.12388417]
 [-0.08670373  0.10614868  0.00561079 -0.0467115   0.12622699  0.06298666
   0.00522941 -0.09001999  0.03938799 -0.07473399  0.07510801 -0.0292979
  -0.14273578  0.10621425 -0.01355848 -0.01956656]]


In [209]:
# tauの方の微分は？
def q1(actor, critic, states, ts, values):
    params = actor.trainable_weights  
    state_input = tf.placeholder(tf.float32, shape=(None, 1, 2))
    ts_input = tf.placeholder(tf.float32, shape=(None, 1))
    values_input = tf.placeholder(tf.float32, shape=(None, 1))
    
    actor_output = actor(state_input)
    discount = tf.expand_dims(- tf.exp(- alpha * actor_output[:,1]), 1)
    weights = tf.multiply(K.exp(- alpha * ts_input), values_input) # param勾配ないならok
    loss2 = tf.matmul(tf.transpose(weights), discount)[0][0]
    gradient_for_d = dummy_optimizer.get_gradients(loss2, params)
    f = K.function([state_input, ts_input, values_input], gradient_for_d)
    return f([states, ts, values])


print(q1(actor, critic, states, ts, values)[3])

def g4one(actor, critic, state):
    params = actor.trainable_weights
    
    state_input = tf.constant([[state.tolist()]], tf.float32)
    actor_output = actor(state_input)
    discount = - tf.exp(- alpha * actor_output[0][1])
    gradient_for_d = dummy_optimizer.get_gradients(discount, params)
    next_state_input = tf.constant([[next_state.tolist()]], tf.float32)
    next_action = actor(next_state_input)
    next_value = critic([next_action, next_state_input])[0][0]
    gradient_for_d = [next_value * g for g in gradient_for_d]
    
    return gradient_for_d

def q2(actor, critic, states, ts, next_states):
    batch_size = len(states)
    for i in range(batch_size):
        state, t = states[i], ts[i][0]
        discount = np.exp(- alpha * t)
        g_tensor = g4one(actor, critic, state)
        g_tensor = [discount * g for g in g_tensor]
        if i == 0:
            out = [tf.constant(0.)] * len(g_tensor)
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
        else:
            out = [K.add(o, g) for o, g in zip(out, g_tensor)]
    func = K.function([], out)
    gg = func([[]])
    return gg

print(q2(actor, critic, states, ts, next_states)[3])


[ 0.34201953 -0.24483807  0.04055472 -0.12173391 -0.21112615 -0.03112669
 -0.01712145 -0.00383606  0.01577214  0.08123913  0.          0.
  0.          0.          0.          0.0236667 ]
[ 0.20547928 -0.14791636  0.02281159 -0.0772059  -0.12995286 -0.01997889
 -0.00961708 -0.0048512   0.00886675  0.04498969  0.          0.
  0.          0.          0.          0.01868356]
